pca_tfidf_250features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings 
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Tfidf/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Tfidf/tfidf_withstopwordchangelevels_250_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,240,241,242,243,244,245,246,247,248,249
0,0,-0.845502,-0.565622,-0.155091,-0.096732,-1.142536,-0.249853,0.170731,-0.268309,-0.132136,...,-0.206628,0.257725,-0.212051,0.532170,-0.389574,0.552421,-0.200591,0.050551,-0.664444,0.219483
1,1,-0.874028,-0.610195,-0.167818,0.107802,-0.910688,0.047421,0.178206,-0.096603,0.134759,...,1.657864,-0.723187,0.332011,-0.454476,1.227931,-0.504784,0.280456,0.662654,0.803215,0.859502
2,2,-0.901905,-0.596085,-0.163331,-0.054099,-1.251360,-0.108977,0.183631,-0.263528,-0.105314,...,0.062744,1.023016,0.617350,-0.901643,0.193856,0.436707,0.244089,0.051315,-0.224830,0.355498
3,3,-1.219312,-1.199603,-0.314512,-1.571484,78.482621,-98.014648,99.594246,-5.530056,-1.043357,...,-0.189275,-0.250393,-0.048211,-0.125194,-0.069413,0.109508,-0.030084,-0.093052,-0.039986,-0.170247
4,4,-0.793535,-0.409023,-0.180451,-0.118322,-1.053552,-0.173959,0.144951,-0.224519,-0.130030,...,0.131049,1.210504,0.340861,0.634931,0.644031,-0.528201,1.584673,0.715468,-0.051545,0.290443


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,-0.845502,-0.565622,-0.155091,-0.096732,-1.142536,-0.249853,0.170731,-0.268309,-0.132136,0.070040,...,-0.206628,0.257725,-0.212051,0.532170,-0.389574,0.552421,-0.200591,0.050551,-0.664444,0.219483
1,-0.874028,-0.610195,-0.167818,0.107802,-0.910688,0.047421,0.178206,-0.096603,0.134759,0.669677,...,1.657864,-0.723187,0.332011,-0.454476,1.227931,-0.504784,0.280456,0.662654,0.803215,0.859502
2,-0.901905,-0.596085,-0.163331,-0.054099,-1.251360,-0.108977,0.183631,-0.263528,-0.105314,0.101580,...,0.062744,1.023016,0.617350,-0.901643,0.193856,0.436707,0.244089,0.051315,-0.224830,0.355498
3,-1.219312,-1.199603,-0.314512,-1.571484,78.482621,-98.014648,99.594246,-5.530056,-1.043357,20.114230,...,-0.189275,-0.250393,-0.048211,-0.125194,-0.069413,0.109508,-0.030084,-0.093052,-0.039986,-0.170247
4,-0.793535,-0.409023,-0.180451,-0.118322,-1.053552,-0.173959,0.144951,-0.224519,-0.130030,0.124931,...,0.131049,1.210504,0.340861,0.634931,0.644031,-0.528201,1.584673,0.715468,-0.051545,0.290443


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 250), (524, 250))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [16]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [17]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3722493406417582


**Decision Tree**

In [18]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [19]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [20]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [21]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [22]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [23]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.440984,0.440984,0.541786,0.420945,0.305344,0.305344,0.366995,0.283488


**Random Forest**

In [24]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [25]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [26]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [27]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9945    0.9945    0.9945       361
           2     0.9982    0.9946    0.9964       553
           3     0.9903    0.9967    0.9935       306

    accuracy                         0.9951      1220
   macro avg     0.9943    0.9953    0.9948      1220
weighted avg     0.9951    0.9951    0.9951      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3649    0.1720    0.2338       157
           2     0.4530    0.7932    0.5767       237
           3     0.2571    0.0692    0.1091       130

    accuracy                         0.4275       524
   macro avg     0.3583    0.3448    0.3065       524
weighted avg     0.3780    0.4275    0.3579       524



In [28]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.440984,0.440984,0.541786,0.420945,0.305344,0.305344,0.366995,0.283488
1,RandomForest,0.995082,0.995082,0.995095,0.995084,0.427481,0.427481,0.378008,0.357935


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:

classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5787    0.3463    0.4333       361
           2     0.5205    0.7559    0.6165       553
           3     0.6468    0.4248    0.5128       306

    accuracy                         0.5516      1220
   macro avg     0.5820    0.5090    0.5209      1220
weighted avg     0.5694    0.5516    0.5363      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3051    0.2293    0.2618       157
           2     0.4566    0.6667    0.5420       237
           3     0.2167    0.1000    0.1368       130

    accuracy                         0.3950       524
   macro avg     0.3261    0.3320    0.3136       524
weighted avg     0.3517    0.3950    0.3575       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.509016,0.509016,0.614903,0.418994,0.408397,0.408397,0.294679,0.293662
1,RandomForest,0.995082,0.995082,0.995082,0.995082,0.417939,0.417939,0.372604,0.358967
2,Adaboost,0.551639,0.551639,0.569414,0.536287,0.395038,0.395038,0.351699,0.357547


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9771    0.9474    0.9620       361
           2     0.9516    0.9946    0.9726       553
           3     0.9932    0.9477    0.9699       306

    accuracy                         0.9689      1220
   macro avg     0.9740    0.9632    0.9682      1220
weighted avg     0.9696    0.9689    0.9688      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3304    0.2357    0.2751       157
           2     0.4355    0.6414    0.5188       237
           3     0.1905    0.0923    0.1244       130

    accuracy                         0.3836       524
   macro avg     0.3188    0.3231    0.3061       524
weighted avg     0.3432    0.3836    0.3479       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.509016,0.509016,0.614903,0.418994,0.408397,0.408397,0.294679,0.293662
1,RandomForest,0.995082,0.995082,0.995082,0.995082,0.417939,0.417939,0.372604,0.358967
2,Adaboost,0.551639,0.551639,0.569414,0.536287,0.395038,0.395038,0.351699,0.357547
3,GBM,0.968852,0.968852,0.969560,0.968789,0.383588,0.383588,0.343223,0.347909


**SVM Model**

In [29]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [30]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [31]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [32]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6090    0.6731    0.6395       361
           2     0.7224    0.6637    0.6918       553
           3     0.6422    0.6569    0.6494       306

    accuracy                         0.6648      1220
   macro avg     0.6579    0.6645    0.6602      1220
weighted avg     0.6687    0.6648    0.6657      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3396    0.4586    0.3902       157
           2     0.4456    0.3629    0.4000       237
           3     0.2521    0.2308    0.2410       130

    accuracy                         0.3588       524
   macro avg     0.3458    0.3507    0.3437       524
weighted avg     0.3658    0.3588    0.3576       524



In [33]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.440984,0.440984,0.541786,0.420945,0.305344,0.305344,0.366995,0.283488
1,RandomForest,0.995082,0.995082,0.995095,0.995084,0.427481,0.427481,0.378008,0.357935
2,SVM,0.664754,0.664754,0.668747,0.665691,0.358779,0.358779,0.365840,0.357621


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4745    0.5402    0.5052       361
           2     0.5798    0.6835    0.6274       553
           3     0.5223    0.2680    0.3542       306

    accuracy                         0.5369      1220
   macro avg     0.5255    0.4972    0.4956      1220
weighted avg     0.5342    0.5369    0.5227      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3187    0.3248    0.3218       157
           2     0.4430    0.5907    0.5063       237
           3     0.3125    0.1154    0.1685       130

    accuracy                         0.3931       524
   macro avg     0.3581    0.3436    0.3322       524
weighted avg     0.3734    0.3931    0.3672       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.509016,0.509016,0.614903,0.418994,0.408397,0.408397,0.294679,0.293662
1,RandomForest,0.995082,0.995082,0.995082,0.995082,0.417939,0.417939,0.372604,0.358967
2,Adaboost,0.551639,0.551639,0.569414,0.536287,0.395038,0.395038,0.351699,0.357547
3,GBM,0.968852,0.968852,0.969560,0.968789,0.383588,0.383588,0.343223,0.347909
4,SVM,0.681148,0.681148,0.716450,0.668473,0.374046,0.374046,0.345484,0.349370
5,KNN,0.536885,0.536885,0.534183,0.522708,0.393130,0.393130,0.373414,0.367228


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6667    0.1163    0.1981       361
           2     0.4755    0.9494    0.6337       553
           3     0.7170    0.1242    0.2117       306

    accuracy                         0.4959      1220
   macro avg     0.6197    0.3966    0.3478      1220
weighted avg     0.5927    0.4959    0.3990      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3448    0.0637    0.1075       157
           2     0.4478    0.8692    0.5911       237
           3     0.2857    0.0769    0.1212       130

    accuracy                         0.4313       524
   macro avg     0.3595    0.3366    0.2733       524
weighted avg     0.3767    0.4313    0.3296       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.509016,0.509016,0.614903,0.418994,0.408397,0.408397,0.294679,0.293662
1,RandomForest,0.995082,0.995082,0.995082,0.995082,0.417939,0.417939,0.372604,0.358967
2,Adaboost,0.551639,0.551639,0.569414,0.536287,0.395038,0.395038,0.351699,0.357547
3,GBM,0.968852,0.968852,0.969560,0.968789,0.383588,0.383588,0.343223,0.347909
4,SVM,0.681148,0.681148,0.716450,0.668473,0.374046,0.374046,0.345484,0.349370
5,KNN,0.536885,0.536885,0.534183,0.522708,0.393130,0.393130,0.373414,0.367228
6,GNB,0.495902,0.495902,0.592654,0.398952,0.431298,0.431298,0.376747,0.329640
